# Wordle Solver

In [1]:
# import functions
from wordle_helpers import *

In [2]:
# help(main)

In [3]:
# Main function that returns a list of possible words
green_letters = ''
green_positions = []
yellow_letters = 'CNHU'
yellow_positions = [1,4,1,3,]
bad_letters = 'RAEORI'

# Main function that returns a list of possible words
words = main(green_letters, green_positions, yellow_letters, yellow_positions, bad_letters)
print(words)

['bunch', 'lunch', 'munch', 'punch']


In [4]:
# Repetitive letters
letters_df = repetitive_letters(wordle_list=words)
print(letters_df.to_string(index=False))

Letters  Count
      U      4
      N      4
      C      4
      H      4
      B      1
      L      1
      M      1
      P      1


#### Choosing next word

In [5]:
# Main function that returns a list of possible words
green_letters = ''
green_positions = []
yellow_letters = 'BLMP'
yellow_positions = []
bad_letters = ''

# Main function that returns a list of possible words
words = main(
    green_letters,
    green_positions,
    yellow_letters,
    yellow_positions,
    bad_letters,
    wordle_list_path="wordle-possible-words",
)
print(words)

['blimp', 'plomb', 'plumb']


In [6]:
# help(score)

In [7]:
df = load_data(file_name='wordle_ranking.csv')
df

,Names,Games_Won
0,Murilo,181
1,Barbara,75
2,Draw,183


In [8]:
# Update score
score_df = score()
score_df

,Names,Games_Won
0,Murilo,181
1,Barbara,75
2,Draw,184


In [9]:
# Print results
print(score_df.to_string(index=False))

  Names  Games_Won
 Murilo        181
Barbara         75
   Draw        184


In [10]:
# help(reset_score)

#### Set custom score

In [11]:
# help(set_score)

In [12]:
# # Set custom score
# df = set_score(m_score=168, b_score=74, draw_score=168)
# df

#### Reset score

In [13]:
## Reset score
# reset_score()